In [ ]:
#%matplotlib notebook
from ABME import *
from agents import *

In [ ]:
def simulate(agents, tracker, gridsize=50, iterations=1000, rate=0.1, radius=10, render=False):
    #Setup objects
    # First create a grid for the model and the model itself
    g = Grid(50);
    m = Model();
    m.dead = []; #Create a list for the dead
    #and add the grid to the model
    m.set_grid(g);

    # Create a visualizer and a tracker
    vis = Visualizer(m);
    tracker.model = m;

    # And track population using the default tracker
#     tracker.add_tracker('Population', lambda m: len(m.agents)); # lambda model: len(model.agents)

    #Same as: def foo(model): return len(model.agents);
    #         tracker.add_tracker('Population', foo);

    #Later, tracker['Population'] can be used to get the population data

    #Create 2 resources, these are automatically added to the grid [g]
    r1 = Resource("Kerbonite", g);
    r2 = Resource("Ebolite", g);

    # Create a capacity function and regrowth function
    cap=lambda x:np.round(4*np.exp(-(x**2)/(2*(0.10)**2))); #  x=distance from center(given in grow)
    reg=lambda x:0.5; # Uniform growth of 0.5, x is ignored

    # Set capacity
    #    scale = true makes the distance from center from [0, 1]
    #    Making a bigger map thus keeps the ratios of resources equal
    #    scale = True is default
#     r1.set_capacity(15,15, fun=cap, scale=True);
#     r1.set_capacity(35,35, fun=cap, scale=True);
#     r2.set_capacity(35,15, fun=cap, scale=True);
#     r2.set_capacity(15,35, fun=cap, scale=True);

#     # grow to the given capacity (such that there are resources available in the first tick)
#     r1.grow(15,15, fun=cap);
#     r2.grow(35,15, fun=cap);
#     r1.grow(35,35, fun=cap);
#     r2.grow(15,35, fun=cap);


    #add agents
    for agent in agents:
        m.add_agent(agent);

    # or add a single test agent
    # a = SugerAgent();
    # #Modify random variables before adding the agent
    # # Modifying other variables which are randomly initialized, like vision, have no effect
    # # before the add_agent call has been made as this call initializes these variables
    # a.wealth_low = 10;
    # a.wealth_high = 11;
    # #Add the agent
    # #    This call also initializes the agents values based on the _low _high
    # #    From nowon, modifying things like wealth_low, has no effect
    # m.add_agent(a, position=[35,35]); 
    # #Set variables of this agent
    # a.max_age = 20000;
    # a.metabolism = 0.1;
    # a.vision = 30;
    vis = Visualizer(m);

    # simulate
    for i in range(iterations):
        # Core loop
        x1 = -radius * np.sin((rate*i)/np.pi) + 25;
        y1 = radius * np.cos((rate*i)/np.pi) + 25;
        x2 = radius * np.sin((rate*i)/np.pi) + 25;
        y2 = -radius * np.cos((rate*i)/np.pi) + 25;
        r1.capacity = None; r2.capacity = None;
        r1.set_capacity(x1,y1, fun=cap);
        r2.set_capacity(x2,y2, fun=cap);
        r1.grow(x1,y1, fun=cap);
        r2.grow(x2,y2, fun=cap);
        # following 3 calls is the same as m.step()
        m.step_move();
        m.step_act();
        m.track(); #Using all trackers, save data

        #Break if all agents have dieded
        if len(m.agents) == 0:
            break;
        
        if render:
            vis.clear();
            fig = vis.plot_grid(new_fig=True);
            pl.show();
            vis.show();
            
    return m;

In [ ]:
def recombinate(g1, g2, mutation_std=0.1):
    #Take average
    ng = (np.array(g1) + np.array(g2)) / 2;
    #add mutation
    for i in range(len(ng)):
        ng[i] = np.random.normal(ng[i], mutation_std);
    
    return ng;

def create_new_gen(population, size, privilaged=10, 
                   self_replication=5, num_mates = 10, offspring_per_mate=3, fill=True, mutation_rate=0.1):
    new_pop = [];
        
    #Sort current population on fitness
    population.sort(key=lambda a: a.fitness(), reverse=True);

    #Number of privilaged members of population, the progenitors of the next generation
    # They will now mate with num_mates
    if privilaged > len(population):
        privilaged = len(population);
    for i in range(privilaged):
        progenitor = population[i];
        for j in range(num_mates):
            mate = np.random.choice(population);
            for k in range(offspring_per_mate):
                new_genome = recombinate(progenitor.genome, mate.genome, mutation_rate);
                child = EvolutionAgent();
                child.genome = new_genome;
                child.vis_metb_factor = progenitor.vis_metb_factor;
                new_pop.append(child);
                
    #Self replication
    for i in range(privilaged):
        a = population[i];
        for j in range(self_replication):
            new_genome = recombinate(a.genome, a.genome, mutation_rate);
            child = EvolutionAgent();
            child.genome = new_genome;
            child.vis_metb_factor = a.vis_metb_factor;
            new_pop.append(child);
    
    #Add random childs till population size is equal again
    for i in range(len(new_pop), size):
        a  = EvolutionAgent();
        a.vis_metb_factor = np.random.choice(population).vis_metb_factor;
        new_pop.append(a);

    return new_pop;

def create_new_gen_abovemean(agents, size, duplicate_survival=5, mutation_rate=0.1, n_random=5):
    #Select all agents above the mean
    meanfit = np.mean([a.fitness() for a in agents]);
    best = [a for a in agents if a.fitness() >= meanfit];
    
    #Create new population with the best and their children
    newpop = [];
    for i in range(duplicate_survival):
        for a in best:
            child = EvolutionAgent();
            child.genome = a.genome;
            child.parent_fitness = a.fitness();
            child.vis_metb_factor = a.vis_metb_factor;
            newpop.append(child);
        if len(newpop) >= (size-len(best)-n_random):
            break;
        
    #Create children
    rest = size - len(newpop) - n_random;
    
    for i in range(rest):
        mate1 = np.random.choice(best);
        mate2 = np.random.choice(best);
        child = EvolutionAgent();
        child.vis_metb_factor = mate1.vis_metb_factor;
        child.genome = recombinate(mate1.genome, mate2.genome, mutation_rate);
        child.parent_fitness = (mate1.fitness() + mate2.fitness())/2;
        newpop.append(child);
        
    for i in range(n_random):
        a  = EvolutionAgent();
        a.vis_metb_factor = np.random.choice(best).vis_metb_factor;
        newpop.append(a);
        
    return newpop;

Create P1, the initial population of agents

In [ ]:
# Settings
N = 1000; #Population size
wealth_low = 5;
wealth_high = 6;
its_per_gen = 1000;
vis_metb_factor = 0.1;

In [ ]:
#Create population, note that you cannot call fitness before they are in the simulation
population = [];
for i in range(N):
    a = EvolutionAgent();
    a.wealth_low = wealth_low;
    a.wealth_high = wealth_high;
    a.vis_metb_factor = vis_metb_factor;
    population.append(a);

Now setup a simulation and keep simulating and creating new offsprings

In [ ]:
sim_tracker = Tracker(); #logs data per sim iteration
evo_tracker = Tracker(); #logs data per evolution

evo_tracker.add_tracker("Fitness", lambda m: [a.fitness() for a in m.agents]);

In [ ]:
mfitness = [];
mxfitness = [];
for generation in range(100):
    display.clear_output(wait=True);
    model = simulate(population, sim_tracker, gridsize=50, iterations=its_per_gen, rate=0.2, radius=14);

    #create new population
    cpop = [];
    for a in model.agents:
        cpop.append(a);
        
    for a in model.dead:
        if (a.age > 3):
            cpop.append(a);
        
    if len(cpop) == 0:
        cpop = model.dead;
        
    print("Generation %i" % generation);
    mfitness.append(np.mean([a.fitness() for a in cpop]));
    mxfitness.append(np.max([a.fitness() for a in cpop]));
    print("Mean fitness: %f (%f)" % (np.mean([a.fitness() for a in cpop]), np.std([a.fitness() for a in cpop])));
    print("Max  fitness: %f" % np.max([a.fitness() for a in cpop]))
    print("%i survived %i iterations" % (len(model.agents),model.tick))
    print("Visions:")
    for i in range(1,np.max([a.vision for a in cpop])+1):
        print("\tvision %i: %i" % (i, sum([a.vision==i for a in cpop])))
    pl.hist([a.vision for a in cpop]);
    pl.title("Histogram of vision");
    pl.xlabel("Vision");
    pl.ylabel("# agents")
        
    #print surviving agents
#     for a in model.agents:
#         print(a);
    
    # Make all agents part of the model again, so we can use visualizer
    # THis ofc only adds dead agens if all agents died
    model.agents = cpop; 

    population = create_new_gen(cpop, N, \
                                mutation_rate=0.01, \
                                privilaged=50, \
                                offspring_per_mate=2, \
                                num_mates=4, \
                               self_replication=4);
#     population = create_new_gen_abovemean(cpop, N, duplicate_survival=2, mutation_rate=0.10, n_random=50);
    
#SHow final generations state
    vis = Visualizer(model);
    vis.plot_grid();
    vis.plot_wealth_hist();
    vis.plot_needs_metabolism();
    pl.figure();
    pl.plot(mfitness);
    pl.plot(mxfitness, 'g');
    pl.title('avg. Fitness + max fitness (green)')
    pl.show();
    vis.show();
    